In [2]:
import sagemaker
sagemaker_session = sagemaker.session.Session()

In [3]:
# Get pre-trained model
style_model = "style_7" # actual filename has .pth

bucket = "stepin2mpiece"
model_dir = "models/"
local_model = style_model + ".pth"
tmp_local_model = "/tmp/"  + local_model
zipped_file = style_model + ".tar.gz"
zipped_local_model = "/tmp/" + style_model + ".tar.gz"
zipped_model_key = model_dir + style_model + ".tar.gz"
model_path = "s3://" + bucket + "/" +zipped_model_key

In [4]:
pre_trained_model = sagemaker_session.download_data(tmp_local_model, bucket, model_dir + style_model + ".pth")

! (cd /tmp; tar -czvf  {zipped_file} {local_model} )

model_uri_pt = sagemaker_session.upload_data(path=zipped_local_model, key_prefix=zipped_model_key)

! rm -rf {local_model} {zipped_local_model}

style_7.pth/
style_7.pth/style_7.pth


In [6]:
# deploy the model with a new endpoint
import os
import boto3
import re
import json
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.model import SKLearnModel
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import time
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

pytorch_model = PyTorchModel(
    model_data= model_path, 
    role=role,
    source_dir="code",
    entry_point='inference.py',
    framework_version='1.8',
    py_version="py3")

endpoint_name = "STEPIN2-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("EndpointName= {}".format(endpoint_name))

predictor = pytorch_model.deploy(initial_instance_count=1, 
                                 instance_type="ml.g4dn.4xlarge", 
                                 endpoint_name=endpoint_name)

arn:aws:iam::921212210452:role/service-role/AmazonSageMaker-ExecutionRole-20221110T154407
EndpointName= STEPIN2-2022-11-19-02-35-50
-----------!